In [1]:
import pandas as pd
import numpy as np
import urllib
import random
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score
import scipy as sp
from sklearn.cross_validation import KFold
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

/Users/danqingwang/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [111]:
# read in datasets
x_test = pd.read_csv('x_test.csv',index_col=0)
x_train = pd.read_csv('x_train.csv',index_col=0)
y_test = pd.read_csv('y_test.csv',index_col=0)
y_train = pd.read_csv('y_train.csv',index_col=0)

In [112]:
# Function f1_genre
# input: two pandas dataframes, 
    # genre_real: predicted values
    # genre_predict: real values
# output: mean f1 score of each class
def f1_genres(genre_real, genre_predict):
    count_row = len(genre_real)
    if count_row == 0:
        print "No data in dataframe!"
        return
    if count_row != len(genre_predict):
        print "Different length of predicted and real dataframes!"
        return
    count_col = len(genre_real.columns)
    if count_col == 0:
        print "No data in dataframe!"
        return
    if count_col != len(genre_predict.columns):
        print "Different genres of predicted and real dataframes!"
        return
    score = 0
    for i in range(count_col):
        score += f1_score(genre_real[genre_real.columns.values[i]], genre_predict[genre_predict.columns.values[i]])
    score = score/count_col
    return(score)

In [113]:
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_np = scaler.transform(x_train)
x_test_np = scaler.transform(x_test)

indexs_train = x_train.index
indexs_test = x_test.index
x_train = pd.DataFrame(x_train_np, index = indexs_train, columns = x_train.columns)
x_test = pd.DataFrame(x_test_np, index = indexs_test, columns = x_test.columns)

## Tuning SVM

In [134]:
genre_pred = pd.DataFrame(index = x_test.index) # dataframe to store predicted values
c = range(-7, 7)

for col in y_train.columns:
    k_cv = 5
    score_cv = []
    for i in range(len(c)):
        # fit regularized logistic regression model on training set      
        svm = SVC(C=10**c[i], class_weight='balanced')
        score_cv += [sum(cross_val_score(svm, x_train, y_train[col], cv = k_cv, scoring = F_score_cv)) / k_cv]
    # find best score and corresponding tuning parameter
    C_best = 10**c[np.argmax(score_cv)]
    print col, C_best
    svm = SVC(class_weight='balanced',  C = C_best)
    svm.fit(x_train, y_train[col])
    genre_pred[col]= svm.predict(x_test)

score_svm = f1_genres(y_test, genre_pred)
print score_svm

Action 1
Adventure 1
Comedy 1
Crime 1
Fantasy 1
Family 1
Romance 1
Horror 1
Western 10
Documentary 1
Biography 1
Drama 1
Animation 1000
Sci-Fi 10
Thriller 1
Short 1
Mystery 1
Sport 100000
War 10
History 1
Music 10
Foreign 1
Other 100000
0.390064260284


In [135]:
print score_svm

0.390064260284


-----